In [9]:
%reload_ext autoreload
%autoreload 2

# ------------------------------------------------------------------------------
# Licensed under the MIT License.
# ------------------------------------------------------------------------------

import os
# os.environ['CUDA_DEVICE_ORDER']='PCI_BUS_ID'
os.environ['CUDA_VISIBLE_DEVICES']="3"


import argparse
from inspect import ArgSpec
import os
import pprint
import shutil
import sys

import random
import logging
import time
import timeit
import ast
from pathlib import Path
from unittest import result

import numpy as np
import copy

import torch
import torch.nn as nn
import torch.backends.cudnn as cudnn
from tensorboardX import SummaryWriter

import tools._init_paths
import models
import dataset
from config import cfg
from config import update_config
from core.criterion import CrossEntropy, OhemCrossEntropy
from core.seg_function import validate_seg as validate
from core.oneshot_function import calib_bn_seg as calib_bn
from utils.utils import get_model_summary
from utils.utils import create_logger, FullModel
from dataset.roidb import combined_roidb_for_training
from roi_data.loader import RoiDataLoader



In [10]:
def parse_args(l):
    parser = argparse.ArgumentParser(description='Train segmentation network')

    parser.add_argument('--cfg',
                        help='experiment configure file name',
                        required=True,
                        type=str)
    parser.add_argument('opts',
                        help="Modify config options using the command-line",
                        default=None,
                        nargs=argparse.REMAINDER)
    parser.add_argument('--bn_calib',
                        action='store_true')
    parser.add_argument('--save_name',
                        type=str,
                        default='oneshot')
    parser.add_argument('--sample_num',
                        type=int,
                        default=100)
    parser.add_argument('--fusion_percentage',
                        type=float,
                        default=0.5)
    parser.add_argument('--depth_list',
                        type=str,
                        default=[2, 3, 4, 5])
    parser.add_argument('--model_batch',
                        type=int,
                        default=3)
    args = parser.parse_args(l)
    update_config(cfg, args)

    return args

arglist = ['--cfg',  'experiments/lesion_Q5/Lesion_Q5_9Slices_scalenet_seg_test.yaml' ,
     '--save_name', 'superscalenet_lesion' ,
     '--sample_num', '6' ,
     '--fusion_percentage' , '0.1' ,
     '--depth_list' ,'[2,3]',
     'TEST.MODEL_FILE' , 'output/Lesion/superscalenet_seg/Lesion_Q5_9Slices_superscalenet/data_patch_train/best.pth']


args = parse_args(arglist)

In [11]:
logger, final_output_dir, tb_log_dir = create_logger(
    cfg, args.cfg, 'test')

writer_dict = {
    'writer': SummaryWriter(tb_log_dir),
    'train_global_steps': 0,
    'valid_global_steps': 0,
}
# cudnn related setting
cudnn.benchmark = cfg.CUDNN.BENCHMARK
cudnn.deterministic = cfg.CUDNN.DETERMINISTIC
cudnn.enabled = cfg.CUDNN.ENABLED

# build model
model = eval('models.' + cfg.MODEL.NAME +
             '.get_seg_model')(cfg)

input_shape = (1, 3, cfg.TEST.IMAGE_SIZE[1], cfg.TEST.IMAGE_SIZE[0])

dump_input = torch.rand(
    (1, 3, cfg.TEST.IMAGE_SIZE[1], cfg.TEST.IMAGE_SIZE[0])
)
# logger.info(get_model_summary(model.cuda(), dump_input.cuda()))

if cfg.TEST.MODEL_FILE:
    model_state_file = cfg.TEST.MODEL_FILE
else:
    model_state_file = os.path.join(final_output_dir,
                                    'final_state.pth')


model.init_weights(model_state_file)
# logger.info('=> loading model from {}'.format(model_state_fil

=> creating output/Lesion/superscalenet_seg/Lesion_Q5_9Slices_scalenet_seg_test/data_patch_test
=> creating log/Lesion/superscalenet_seg/data_patch_test/Lesion_Q5_9Slices_scalenet_seg_test_2022-03-20-15-42


=> init weights from normal distribution
=> init weights from normal distribution
=> loading pretrained model output/Lesion/superscalenet_seg/Lesion_Q5_9Slices_superscalenet/data_patch_train/best.pth
Missing keys in pretrained_dict: set()


In [12]:
model

SuperScaleNet_Seg(
  (conv1): Conv2d(9, 64, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.01, affine=True, track_running_stats=True)
  (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
  (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.01, affine=True, track_running_stats=True)
  (relu): ReLU(inplace)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.01, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.01, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.01, affine=True, track_running_stats=True)
      (relu): ReLU(inpla

In [ ]:
if cfg.LOSS.USE_OHEM:
    criterion = OhemCrossEntropy(ignore_label=cfg.TRAIN.IGNORE_LABEL,
                                 thres=cfg.LOSS.OHEMTHRES,
                                 min_kept=cfg.LOSS.OHEMKEEP
                                 )
else:
    criterion = CrossEntropy(ignore_label=cfg.TRAIN.IGNORE_LABEL,
                             weight=test_dataset.class_weights.cuda())

gpus = list(cfg.GPUS)
print("use gpus of " , gpus)
model = FullModel(model, criterion)

model_list = []
for j in range(args.model_batch):
    if j == 0:
        model_list.append( nn.DataParallel(model, device_ids=gpus).cuda() )
    else:
        model_list.append( nn.DataParallel( copy.deepcopy(model) , device_ids=gpus).cuda() )

In [ ]:
for j in range(40):
    
    model_list.append( nn.DataParallel( copy.deepcopy(model) , device_ids=gpus).cuda() )
    print(j , len(model_list) )

In [ ]:
model_list.pop()
print(j , len(model_list) )

In [ ]:
model_list

In [6]:
(seed , params , flops)  = 1,40000000,7.7
results = []
results.append((seed , params , flops))
results.append((seed+1 , params , flops))
mIoUs = [.3,.8]

In [7]:
print(results,mIoUs)

for (seed , params , flops) , mean_IoU in zip(results , mIoUs):
    print('Model seed {}: params {:.2f} M, flops {:.2f} G AP {})'
                    .format(seed, params / 1e6, flops, mean_IoU))

[(1, 40000000, 7.7), (2, 40000000, 7.7)] [0.3, 0.8]
Model seed 1: params 40.00 M, flops 7.70 G AP 0.3)
Model seed 2: params 40.00 M, flops 7.70 G AP 0.8)


In [8]:
model

NameError: name 'model' is not defined